In [1]:
from google.colab import drive
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
%%bash
cd /content/drive/MyDrive/cv_project_pytorch_yolo
pwd

/content/drive/MyDrive/cv_project_pytorch_yolo


In [5]:
cd /content/drive/MyDrive/cv_project_pytorch_yolo/CV_final_project/weights

/content/drive/.shortcut-targets-by-id/1Ap0A3HuEQConBe5WJXecT75OdJT8dVDa/cv_project_pytorch_yolo/CV_final_project/weights


In [ ]:
# !bash download_yolo_weights.sh

In [6]:
cd /content/drive/MyDrive/cv_project_pytorch_yolo/CV_final_project


/content/drive/.shortcut-targets-by-id/1Ap0A3HuEQConBe5WJXecT75OdJT8dVDa/cv_project_pytorch_yolo/CV_final_project


In [7]:
!python Run_with_2D.py

Using model weights : exp_6_epoch_10.pkl
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth
100% 548M/548M [00:01<00:00, 340MB/s]


In [8]:
!python Eval.py



Generating evaluation results for experiment No.  6
/content/drive/.shortcut-targets-by-id/1Ap0A3HuEQConBe5WJXecT75OdJT8dVDa/cv_project_pytorch_yolo/CV_final_project/weights/exp_6/
Evaluating for Epoch:  10
Loading model with exp_6_epoch_10.pkl
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Length of eval data 0
Model is commencing predictions.....
0it [00:00, ?it/s]
Finished.


In [9]:
!pip install shapely

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
#VGG
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from shapely.geometry import Polygon

def process_file(file_path):
    with open(file_path, 'r') as f:
        data = f.read()

    lines = data.split('\n')
    boxes = []
    for line in lines:
        elements = line.split()
        if len(elements) >= 15 and elements[0] == 'Car':
            x, y, z = float(elements[11]), float(elements[12]), float(elements[13])
            l, w, h = float(elements[8]), float(elements[9]), float(elements[10])
            rotation_y = float(elements[14])
            boxes.append((x, y, z, l, w, h, rotation_y))

    return boxes


def compute_iou(box1, box2):
    # print ("box1", box1)
    # print("box2", box2)
    x1, z1, l1, w1, rot_y1 = box1
    x2, z2, l2, w2, rot_y2 = box2

    r1 = np.array([[np.cos(rot_y1), -np.sin(rot_y1)], [np.sin(rot_y1), np.cos(rot_y1)]])
    r2 = np.array([[np.cos(rot_y2), -np.sin(rot_y2)], [np.sin(rot_y2), np.cos(rot_y2)]])

    box1_corners = np.array([[-l1 / 2, -w1 / 2], [-l1 / 2, w1 / 2], [l1 / 2, w1 / 2], [l1 / 2, -w1 / 2]]).T
    box2_corners = np.array([[-l2 / 2, -w2 / 2], [-l2 / 2, w2 / 2], [l2 / 2, w2 / 2], [l2 / 2, -w2 / 2]]).T

    box1_rotated_corners = r1 @ box1_corners + np.array([[x1], [z1]])
    box2_rotated_corners = r2 @ box2_corners + np.array([[x2], [z2]])

    poly1 = Polygon(box1_rotated_corners.T)
    poly2 = Polygon(box2_rotated_corners.T)

    intersection = poly1.intersection(poly2)
    if intersection.is_empty:
        return 0.0

    iou = intersection.area / (poly1.area + poly2.area - intersection.area)
    return iou


def plot_top_view_boxes(ground_truth_file_path, predicted_file_path):
    ground_truth_boxes = process_file(ground_truth_file_path)
    predicted_boxes = process_file(predicted_file_path)

    fig, ax = plt.subplots(figsize=(10, 10))

    for box in ground_truth_boxes:
        x, y, z, l, w, h, rotation_y = box
        rect = patches.Rectangle((x - l / 2, z - w / 2), l, w, angle=-rotation_y * 180 / 3.14159265, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    for box in predicted_boxes:
        x, y, z, l, w, h, rotation_y = box
        rect = patches.Rectangle((x - l / 2, z - w / 2), l, w, angle=-rotation_y * 180 / 3.14159265, linewidth=1, edgecolor='b', facecolor='none')
        ax.add_patch(rect)

    ious = []
    for gt_box in ground_truth_boxes:
        max_iou = 0
        for pred_box in predicted_boxes:
            iou = compute_iou(gt_box[:5], pred_box[:5])
            max_iou = max(max_iou, iou)
        ious.append(max_iou)

    mean_iou = np.mean(ious)
    print(f"Mean IoU: {mean_iou:.4f}")

    ax.set_xlim(-25, 25)
    ax.set_ylim(0, 50)
    ax.set_aspect('equal')
    ax.set_xlabel('X-axis (left-right)')
    ax.set_ylabel('Z-axis (forward-backward)')
    ax.set_title('Top view of cars (Ground truth: Red, Predicted: Blue)')
    plt.gca().invert_xaxis()
    plt.gca().invert_yaxis()
    return fig

ground_truth_folder = '/content/drive/MyDrive/cv_project_pytorch_yolo/CV_final_project/kitti/validation/label_2'
predicted_folder = '/content/drive/MyDrive/cv_project_pytorch_yolo/CV_final_project/Kitti/results/validation/labels/exp_6/epoch_10'
output_folder = '/content/drive/MyDrive/cv_project_pytorch_yolo/CV_final_project/Kitti/results/validation/birds_eye_view/exp_6'
# print('hi...')

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    # print('hello1')

for file_name in os.listdir(predicted_folder):
    if file_name.endswith('.txt'):
        # print("hello")
        ground_truth_file_path = os.path.join(ground_truth_folder, file_name)
        predicted_file_path = os.path.join(predicted_folder, file_name)
        output_file_path = os.path.join(output_folder, file_name[:-4] + '.png')
        fig = plot_top_view_boxes(ground_truth_file_path, predicted_file_path)
        fig.savefig(output_file_path, bbox_inches='tight')
        plt.close(fig)


Mean IoU: 0.5290
Mean IoU: 0.3735
Mean IoU: 0.2977
Mean IoU: 0.5060
Mean IoU: 0.5041
Mean IoU: nan
Mean IoU: 0.4553
Mean IoU: 0.4567
Mean IoU: 0.6461
Mean IoU: 0.1113


In [13]:
#Mobilenet
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from shapely.geometry import Polygon

def process_file(file_path):
    with open(file_path, 'r') as f:
        data = f.read()

    lines = data.split('\n')
    boxes = []
    for line in lines:
        elements = line.split()
        if len(elements) >= 15 and elements[0] == 'Car':
            x, y, z = float(elements[11]), float(elements[12]), float(elements[13])
            l, w, h = float(elements[8]), float(elements[9]), float(elements[10])
            rotation_y = float(elements[14])
            boxes.append((x, y, z, l, w, h, rotation_y))

    return boxes


def compute_iou(box1, box2):
    # print ("box1", box1)
    # print("box2", box2)
    x1, z1, l1, w1, rot_y1 = box1
    x2, z2, l2, w2, rot_y2 = box2

    r1 = np.array([[np.cos(rot_y1), -np.sin(rot_y1)], [np.sin(rot_y1), np.cos(rot_y1)]])
    r2 = np.array([[np.cos(rot_y2), -np.sin(rot_y2)], [np.sin(rot_y2), np.cos(rot_y2)]])

    box1_corners = np.array([[-l1 / 2, -w1 / 2], [-l1 / 2, w1 / 2], [l1 / 2, w1 / 2], [l1 / 2, -w1 / 2]]).T
    box2_corners = np.array([[-l2 / 2, -w2 / 2], [-l2 / 2, w2 / 2], [l2 / 2, w2 / 2], [l2 / 2, -w2 / 2]]).T

    box1_rotated_corners = r1 @ box1_corners + np.array([[x1], [z1]])
    box2_rotated_corners = r2 @ box2_corners + np.array([[x2], [z2]])

    poly1 = Polygon(box1_rotated_corners.T)
    poly2 = Polygon(box2_rotated_corners.T)

    intersection = poly1.intersection(poly2)
    if intersection.is_empty:
        return 0.0

    iou = intersection.area / (poly1.area + poly2.area - intersection.area)
    return iou


def plot_top_view_boxes(ground_truth_file_path, predicted_file_path):
    ground_truth_boxes = process_file(ground_truth_file_path)
    predicted_boxes = process_file(predicted_file_path)

    fig, ax = plt.subplots(figsize=(10, 10))

    for box in ground_truth_boxes:
        x, y, z, l, w, h, rotation_y = box
        rect = patches.Rectangle((x - l / 2, z - w / 2), l, w, angle=-rotation_y * 180 / 3.14159265, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    for box in predicted_boxes:
        x, y, z, l, w, h, rotation_y = box
        rect = patches.Rectangle((x - l / 2, z - w / 2), l, w, angle=-rotation_y * 180 / 3.14159265, linewidth=1, edgecolor='b', facecolor='none')
        ax.add_patch(rect)

    ious = []
    for gt_box in ground_truth_boxes:
        max_iou = 0
        for pred_box in predicted_boxes:
            iou = compute_iou(gt_box[:5], pred_box[:5])
            max_iou = max(max_iou, iou)
        ious.append(max_iou)

    mean_iou = np.mean(ious)
    print(f"Mean IoU: {mean_iou:.4f}")

    ax.set_xlim(-25, 25)
    ax.set_ylim(0, 50)
    ax.set_aspect('equal')
    ax.set_xlabel('X-axis (left-right)')
    ax.set_ylabel('Z-axis (forward-backward)')
    ax.set_title('Top view of cars (Ground truth: Red, Predicted: Blue)')
    plt.gca().invert_xaxis()
    plt.gca().invert_yaxis()
    return fig

ground_truth_folder = '/content/drive/MyDrive/cv_project_pytorch_yolo/CV_final_project/kitti/validation/label_2'
predicted_folder = '/content/drive/MyDrive/cv_project_pytorch_yolo/CV_final_project/Kitti/results/validation/labels/exp_7/epoch_10'
output_folder = '/content/drive/MyDrive/cv_project_pytorch_yolo/CV_final_project/Kitti/results/validation/birds_eye_view/exp_7'
# print('hi...')

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    # print('hello1')

for file_name in os.listdir(predicted_folder):
    if file_name.endswith('.txt'):
        # print("hello")
        ground_truth_file_path = os.path.join(ground_truth_folder, file_name)
        predicted_file_path = os.path.join(predicted_folder, file_name)
        output_file_path = os.path.join(output_folder, file_name[:-4] + '.png')
        fig = plot_top_view_boxes(ground_truth_file_path, predicted_file_path)
        fig.savefig(output_file_path, bbox_inches='tight')
        plt.close(fig)

Mean IoU: 0.5435
Mean IoU: 0.4581
Mean IoU: 0.3935
Mean IoU: 0.4745
Mean IoU: 0.4755
Mean IoU: nan
Mean IoU: 0.4806
Mean IoU: 0.4794
Mean IoU: 0.5274
Mean IoU: 0.1200
